In [ ]:
!pip install -U scikit-learn==1.3.2 xgboost==1.7.6

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from tqdm import tqdm

# 1. 데이터 로드 및 전처리
df = pd.read_csv('/content/2020_2023_최종데이터.csv')
if 'tm_dt' in df.columns:
    df.drop(columns=['tm_dt'], inplace=True)

# 날짜 파생 변수 생성
df['year'] = df['tm'] // 10000
df['month'] = (df['tm'] % 10000) // 100
df['day'] = df['tm'] % 100
df['weekday'] = pd.to_datetime(df['tm'], format='%Y%m%d').dt.weekday
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)
df['is_before_holiday'] = df['공휴일'].shift(-1, fill_value=0)
df['is_after_holiday'] = df['공휴일'].shift(1, fill_value=0)

# 인코딩 및 평균 타겟 인코딩
encoder = LabelEncoder()
df['address_city'] = encoder.fit_transform(df['address_city'])
df['address_gu_mean_target'] = df.groupby('address_gu')['call_count'].transform('mean')
df['sub_address_mean_target'] = df.groupby('sub_address')['call_count'].transform('mean')

# 데이터 분리
dong_based_gu = {'연제구', '영도구'}
address_gu_all = df['address_gu']
X = df.drop(columns=['tm', 'call_count', 'address_gu', 'sub_address'])
y = df['call_count']
X = X.astype('float32')
y = y.astype('float32')

X_train, X_val, y_train, y_val, gu_train, gu_val = train_test_split(
    X, y, address_gu_all, test_size=0.2, random_state=42
)

# 모델 저장소
gu_models = {}
dong_models = {}

# 모델 학습
gu_list = gu_train.unique()
for gu in gu_list:
    if gu in dong_based_gu:
        sub_list = df[df['address_gu'] == gu]['sub_address'].unique()
        for dong in sub_list:
            mask = (gu_train == gu) & (df['sub_address'] == dong)
            X_train_d = X_train[mask]
            y_train_d = y_train[mask]

            mask_val = (gu_val == gu) & (df['sub_address'] == dong)
            X_val_d = X_val[mask_val]
            y_val_d = y_val[mask_val]

            if len(X_train_d) < 30:
                print(f"⚠️ {gu}-{dong}: 데이터 부족")
                continue

            model = XGBRegressor(n_estimators=300, max_depth=6, learning_rate=0.05,
                                 subsample=0.8, colsample_bytree=0.8, tree_method='hist',
                                 random_state=42)
            model.fit(X_train_d, y_train_d,
                      eval_set=[(X_val_d, y_val_d)],
                      early_stopping_rounds=30,
                      verbose=False)
            rmse = np.sqrt(mean_squared_error(y_val_d, model.predict(X_val_d)))
            dong_models[f"{gu}_{dong}"] = model
            print(f"✅ 동 모델 학습 완료: {gu}-{dong}, RMSE: {rmse:.4f}")
    else:
        mask = gu_train == gu
        X_train_g = X_train[mask]
        y_train_g = y_train[mask]

        mask_val = gu_val == gu
        X_val_g = X_val[mask_val]
        y_val_g = y_val[mask_val]

        if len(X_train_g) < 30:
            print(f"⚠️ {gu}: 데이터 부족")
            continue

        model = XGBRegressor(n_estimators=300, max_depth=6, learning_rate=0.05,
                             subsample=0.8, colsample_bytree=0.8, tree_method='hist',
                             random_state=42)
        model.fit(X_train_g, y_train_g,
                  eval_set=[(X_val_g, y_val_g)],
                  early_stopping_rounds=30,
                  verbose=False)
        rmse = np.sqrt(mean_squared_error(y_val_g, model.predict(X_val_g)))
        gu_models[gu] = model
        print(f"✅ 구 모델 학습 완료: {gu}, RMSE: {rmse:.4f}")

# 11. 전체 검증 RMSE 출력
all_preds = []
all_targets = []

for gu in gu_models.keys():
    X_val_gu = X_val[gu_val == gu]
    y_val_gu = y_val[gu_val == gu]
    y_pred_val = gu_models[gu].predict(X_val_gu)

    all_preds.extend(y_pred_val)
    all_targets.extend(y_val_gu)

total_rmse = np.sqrt(mean_squared_error(all_targets, all_preds))
print(f"\n📊 전체 RMSE (call_count 기준): {total_rmse:.4f}")



/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


✅ 구 모델 학습 완료: 해운대구, RMSE: 1.6321


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


✅ 구 모델 학습 완료: 사하구, RMSE: 1.3843


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

✅ 동 모델 학습 완료: 영도구-청학동, RMSE: 7.9074


/tmp/ipython-input-2-4238224654.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_train_d = X_train[mask]
/tmp/ipython-input-2-4238224654.py:56: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_val_d = X_val[mask_val]
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


✅ 동 모델 학습 완료: 영도구-영선동, RMSE: 2.1428


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


✅ 구 모델 학습 완료: 서구, RMSE: 0.6766

📊 전체 RMSE (call_count 기준): 1.2342


In [ ]:
# 예측
valid_df = pd.read_csv('/content/2024_최종데이터.csv')
if 'tm_dt' in valid_df.columns:
    valid_df.drop(columns=['tm_dt'], inplace=True)

# 평균값 매핑
address_gu_mean_map = df.groupby('address_gu')['call_count'].mean().to_dict()
sub_address_mean_map = df.groupby('sub_address')['call_count'].mean().to_dict()

result_df_list = []

for gu in tqdm(valid_df['address_gu'].unique()):
    gu_df = valid_df[valid_df['address_gu'] == gu]

    if gu in dong_based_gu:
        for dong in gu_df['sub_address'].unique():
            key = f"{gu}_{dong}"
            if key not in dong_models:
                print(f"❌ 모델 없음: {key}")
                continue
            dong_df = gu_df[gu_df['sub_address'] == dong].copy()

            # 파생 변수 및 인코딩
            dong_df['year'] = dong_df['tm'] // 10000
            dong_df['month'] = (dong_df['tm'] % 10000) // 100
            dong_df['day'] = dong_df['tm'] % 100
            dong_df['weekday'] = pd.to_datetime(dong_df['tm'], format='%Y%m%d').dt.weekday
            dong_df['is_weekend'] = dong_df['weekday'].isin([5, 6]).astype(int)
            dong_df['is_before_holiday'] = dong_df['공휴일'].shift(-1, fill_value=0)
            dong_df['is_after_holiday'] = dong_df['공휴일'].shift(1, fill_value=0)

            dong_df['address_city'] = dong_df['address_city'].apply(
                lambda x: x if x in encoder.classes_ else 'Unknown')
            if 'Unknown' not in encoder.classes_:
                encoder.classes_ = np.append(encoder.classes_, 'Unknown')
            dong_df['address_city'] = encoder.transform(dong_df['address_city'])

            dong_df['address_gu_mean_target'] = address_gu_mean_map.get(gu, df['call_count'].mean())
            dong_df['sub_address_mean_target'] = dong_df['sub_address'].map(sub_address_mean_map)
            dong_df['sub_address_mean_target'] = dong_df['sub_address_mean_target'].fillna(df['call_count'].mean())

            X_valid = dong_df.drop(columns=['tm', 'call_count', 'address_gu', 'sub_address'], errors='ignore')
            for col in X.columns:
                if col not in X_valid.columns:
                    X_valid[col] = 0.0
            X_valid = X_valid[X.columns].astype('float32')

            preds = np.round(dong_models[key].predict(X_valid)).astype(int)
            dong_df['call_count'] = preds
            result_df_list.append(dong_df[['tm', 'address_city', 'address_gu', 'sub_address', 'call_count']])
    else:
        if gu not in gu_models:
            continue
        gu_df = gu_df.copy()
        gu_df['year'] = gu_df['tm'] // 10000
        gu_df['month'] = (gu_df['tm'] % 10000) // 100
        gu_df['day'] = gu_df['tm'] % 100
        gu_df['weekday'] = pd.to_datetime(gu_df['tm'], format='%Y%m%d').dt.weekday
        gu_df['is_weekend'] = gu_df['weekday'].isin([5, 6]).astype(int)
        gu_df['is_before_holiday'] = gu_df['공휴일'].shift(-1, fill_value=0)
        gu_df['is_after_holiday'] = gu_df['공휴일'].shift(1, fill_value=0)

        gu_df['address_city'] = gu_df['address_city'].apply(
            lambda x: x if x in encoder.classes_ else 'Unknown')
        if 'Unknown' not in encoder.classes_:
            encoder.classes_ = np.append(encoder.classes_, 'Unknown')
        gu_df['address_city'] = encoder.transform(gu_df['address_city'])

        gu_df['address_gu_mean_target'] = address_gu_mean_map.get(gu, df['call_count'].mean())
        gu_df['sub_address_mean_target'] = gu_df['sub_address'].map(sub_address_mean_map)
        gu_df['sub_address_mean_target'] = gu_df['sub_address_mean_target'].fillna(df['call_count'].mean())

        X_valid = gu_df.drop(columns=['tm', 'call_count', 'address_gu', 'sub_address'], errors='ignore')
        for col in X.columns:
            if col not in X_valid.columns:
                X_valid[col] = 0.0
        X_valid = X_valid[X.columns].astype('float32')

        preds = np.round(gu_models[gu].predict(X_valid)).astype(int)
        gu_df['call_count'] = preds
        result_df_list.append(gu_df[['tm', 'address_city', 'address_gu', 'sub_address', 'call_count']])

100%|██████████| 16/16 [00:00<00:00, 29.04it/s]


In [ ]:

# 제출 파일 생성
valid_preds_df = pd.concat(result_df_list, ignore_index=True)
valid_preds_df.rename(columns={'tm': 'TM'}, inplace=True)

submission_template = pd.read_csv('/content/test_call119.csv', encoding='euc-kr')
submission_template['address_city'] = submission_template['address_city'].apply(
    lambda x: x if x in encoder.classes_ else 'Unknown')
if 'Unknown' not in encoder.classes_:
    encoder.classes_ = np.append(encoder.classes_, 'Unknown')
submission_template['address_city'] = encoder.transform(submission_template['address_city'])
submission_template.drop(columns=['call_count'], inplace=True, errors='ignore')

submission = submission_template.merge(
    valid_preds_df,
    on=['TM', 'address_city', 'address_gu', 'sub_address'],
    how='left'
)

submission['call_count'] = submission['call_count'].fillna(0).astype(int)
submission.to_csv('250028.csv', index=False, encoding='cp949')
print("✅ 제출 파일 저장 완료: 250028.csv")


✅ 제출 파일 저장 완료: 250028.csv


In [ ]:
submission['call_count'].value_counts()

,count
call_count,
1,4504
2,4206
3,508
4,121
0,85
6,56
7,40
5,39
8,30
